# Trabajo Práctico Nro. 2A
### Ing. Javier Ouret - 2025 - UCA - Facultad de Ingeniería
## Monitoreo de dispositivos y software en redes cliente-servidor utilizando SNMP (Entrega 26/05/2025)

#### Activar y configurar SNMP en Ubuntu

Configurar el archivo de SNMP

#### Completar la instalación de SNMP para python
#### Consultar el equipo con ubuntu usando python y con algún gestor snmp gratuito con interfaz web

Si está snmpd activado en Ubuntu, usar pysnmp desde otro equipo (o el mismo) para consultarlo.   
Instalar pysnmp
**NOTA: Directorios indicados son sólo a modo de ejemplo**

Script Python básico (GET):

In [1]:
!which python
import sys
print(sys.executable)

/home/javier/Documents/GitRepo/UCA/PdI/Notebooks/proyecto_snmp/snmp_env/bin/python
/home/javier/Documents/GitRepo/UCA/PdI/Notebooks/proyecto_snmp/snmp_env/bin/python


In [2]:
import pysnmp.hlapi
print("pysnmp funciona")

pysnmp funciona


In [3]:
from pysnmp.entity.engine import SnmpEngine
from pysnmp.hlapi import getCmd, CommunityData, UdpTransportTarget, ContextData, ObjectType, ObjectIdentity

iterator = getCmd(
    SnmpEngine(),
    CommunityData('public'),
    UdpTransportTarget(('127.0.0.1', 161)),
    ContextData(),
    ObjectType(ObjectIdentity('1.3.6.1.2.1.1.1.0'))  # sysDescr
)

errorIndication, errorStatus, errorIndex, varBinds = next(iterator)

if errorIndication:
    print(f"Error: {errorIndication}")
elif errorStatus:
    print(f"Error: {errorStatus.prettyPrint()}")
else:
    for varBind in varBinds:
        print(f"{varBind[0]} = {varBind[1]}")


1.3.6.1.2.1.1.1.0 = Linux javier-Yoga-7-16ARP8 6.8.0-59-generic #61~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Apr 15 17:03:15 UTC 2 x86_64


##### Instalar un gestor SNMP gratuitos con interfaz web

- Cacti
  - Enfocado en gráficas SNMP (CPU, memoria, interfaces).
  - Más liviano que LibreNMS.
- Instalación en Ubuntu:

Acceder a: http://localhost/cacti
Tener snmpd activo en el servidor Ubuntu a monitorear.

#### Crear un dispositivo virtual SNMP en Python que permita simular el uso de GET, SET, PUT, etc.   
Utilizar la librería pysnmp, que permite manejar la gestión de SNMP.   
Crear un dispositivo virtual que pueda responder a solicitudes SNMP simuladas.   
Ejemplo:

Servidor SNMP virtual.
Este script configura un servidor SNMP en el puerto 161 para escuchar las solicitudes de GET, SET, etc.

In [12]:
from pysnmp.entity import engine, config
from pysnmp.carrier.asyncore.dgram import udp

print("Iniciando prueba de SNMP")

snmpEngine = engine.SnmpEngine()

config.addV1System(snmpEngine, 'public-read', 'public')

try:
    config.addTransport(
        snmpEngine,
        udp.domainName,
        udp.UdpTransport().openServerMode(('127.0.0.1', 1161))
    )
    print("Puerto 1161 abierto con éxito")
except Exception as e:
    print(f"Error abriendo puerto: {e}")


Iniciando prueba de SNMP
Puerto 1161 abierto con éxito


In [1]:
%%writefile snmp_server.py
from pysnmp.entity import engine, config
from pysnmp.carrier.asyncore.dgram import udp
from pysnmp.entity.rfc3413 import cmdrsp, context
from pysnmp.smi import builder, instrum
import asyncore
import os
from pysnmp.smi import builder as mibbuilder_module

# Controlador con logging para mostrar operaciones GET y SET
class LoggingMibInstrumController(instrum.MibInstrumController):
    def readVars(self, varBinds, acInfo=(None, None)):
        print("Recibiendo GET para:")
        for oid, val in varBinds:
            print(f"  OID: {oid.prettyPrint()}")
        return super().readVars(varBinds, acInfo)

    def writeVars(self, varBinds, acInfo=(None, None)):
        print("Recibiendo SET para:")
        for oid, val in varBinds:
            print(f"  OID: {oid.prettyPrint()} Valor nuevo: {val.prettyPrint()}")
        return super().writeVars(varBinds, acInfo)

def main():
    snmpEngine = engine.SnmpEngine()

    # Configurar comunidad 'public' para lectura
    config.addV1System(snmpEngine, 'public-read', 'public')

    # Abrir puerto UDP 1161 en localhost
    config.addTransport(
        snmpEngine,
        udp.domainName,
        udp.UdpTransport().openServerMode(('0.0.0.0', 1161))

    )

    mibBuilder = snmpEngine.getMibBuilder()

    # Añadir carpeta de MIBs instaladas con pysnmp-mibs
    mib_path = os.path.join(os.path.dirname(mibbuilder_module.__file__), 'mibs')
    mibBuilder.addMibSources(builder.DirMibSource(mib_path))

    # Cargar MIB IF-MIB
    mibBuilder.loadModules('IF-MIB')

    # Crear controlador con logging
    mibInstrum = LoggingMibInstrumController(mibBuilder)

    # Crear contexto SNMP sin parámetros (crea contexto por defecto)
    snmpContext = context.SnmpContext(snmpEngine)

    # Reemplazar el controlador de instrumentación por defecto con el personalizado
    snmpContext._mibInstrum = mibInstrum

    # Registrar los command responders para manejar GET, SET, etc.
    cmdrsp.GetCommandResponder(snmpEngine, snmpContext)
    cmdrsp.SetCommandResponder(snmpEngine, snmpContext)
    cmdrsp.NextCommandResponder(snmpEngine, snmpContext)
    cmdrsp.BulkCommandResponder(snmpEngine, snmpContext)

    print("SNMP agent corriendo en puerto 1161 (localhost)")

    try:
        snmpEngine.transportDispatcher.jobStarted(1)  # Esto evita que el dispatcher se detenga
        snmpEngine.transportDispatcher.runDispatcher()
    except KeyboardInterrupt:
        print("\nSNMP Agent detenido")
        snmpEngine.transportDispatcher.closeDispatcher()

if __name__ == '__main__':
    main()


Overwriting snmp_server.py


- MIB Simulada (SimpleMIB): Esta clase almacena los valores de los OIDs simulados. En este caso, hemos definido un solo OID, que es el estado de un dispositivo (con un valor de 1, indicando "ON").
- Operaciones GET y SET: La función handle_get_request() simula una solicitud GET, y la función handle_set_request() simula una solicitud SET.
- Servidor SNMP: El servidor SNMP virtual se configura para escuchar en el puerto 161 (el puerto estándar para SNMP). En este ejemplo, está diseñado para simular respuestas GET y SET.

##### Simular un Cliente SNMP

Usar un cliente SNMP para realizar solicitudes como GET, SET, PUT, etc. 
Este código consulta el servidor SNMP para obtener el valor del OID definido anteriormente.

In [3]:
from pysnmp.hlapi import *

def snmp_get(oid):
    iterator = getCmd(
        SnmpEngine(),
        CommunityData('public', mpModel=0),
        UdpTransportTarget(('localhost', 1161)),
        ContextData(),
        ObjectType(ObjectIdentity(oid))
    )

    error_indication, error_status, error_index, var_binds = next(iterator)

    if error_indication:
        print(f"SNMP GET Error: {error_indication}")
    elif error_status:
        print(f"SNMP GET Error Status: {error_status}")
    else:
        for var_bind in var_binds:
            print(f"GET Response: {var_bind}")

# Realizar un GET
snmp_get('1.3.6.1.2.1.2.2.1.7.2')


SNMP GET Error: No SNMP response received before timeout


- Simulación de SET: Para realizar un SET, usar un código similar al de GET, pero utilizando la función setCmd en lugar de getCmd.

- Simular dispositivos más complejos: agregar más OIDs a la clase SimpleMIB para simular dispositivos más complejos con múltiples variables.

##### Herramientas útiles para trabajar con MIBs
- snmptranslate: para ver los OID en texto y número.
- snmpwalk/snmpget: para consultar datos reales con SNMP.
- Zabbix/Nagios/PRTG: para integrarlas a sistemas de monitoreo.
- MIB Browsers: iReasoning o SNMP MIB Browser de ManageEngine.

#### Descargar MIB Browser para Windows y para Linux

https://www.ireasoning.com/mibbrowser.shtml

#### Ejercicio: Instalar un agente SNMP en Windows
- En Windows instalar SNMP y activarlo con PowerShell como administrador

Configuración básica:

Set-ItemProperty -Path "HKLM:\SYSTEM\CurrentControlSet\services\SNMP\Parameters\RFC1156Agent" -Name "sysContact" -Value "Nombre" -type String

Set-ItemProperty -Path "HKLM:\SYSTEM\CurrentControlSet\services\SNMP\Parameters\RFC1156Agent" -Name "sysLocation" -Value "UCA" -type String

Set-ItemProperty -Path "HKLM:\SYSTEM\CurrentControlSet\Services\SNMP\Parameters\ValidCommunities" -Name "COMUNIDAD_PDI" -Value 8 -type DWord

Set-ItemProperty -Path "HKLM:\SYSTEM\CurrentControlSet\Services\SNMP\Parameters\PermittedManagers" -Name "1" -Value "localhost" -type String

Set-ItemProperty -Path "HKLM:\SYSTEM\CurrentControlSet\Services\SNMP\Parameters\PermittedManagers" -Name "2" -Value "192.168.1.78" -type String

Restart-Service -Name SNMP


Mofidicar la configuración del MIB Browser

En address agregar "localhost"

![image](https://github.com/jaouret/PDI_2024-TP1-TP2-TP3-TP4/assets/111520053/ad5e4c2c-04a2-4dc2-830c-57262cdfb967)


En advanced en los campos read community y write community agregar COMUNIDAD_PDI. Y la version 2
![image](https://github.com/jaouret/PDI_2024-TP1-TP2-TP3-TP4/assets/111520053/36b36bb8-9f46-438b-87fd-944f64ab80c7)

#### Ejercicio: Instalar un agente SNMP en Linux

Ejecutar el MIB Browser sobre Windows o Linux.   
Describir lo que se observa en un informe.

#### Pequeña aplicacion web flask para consultar parametros con snmp como sysuptime, etc.
Permite consultar parámetros SNMP como sysUptime, sysDescr, etc., usando pysnmp.

Código de la app Flask (snmp_web.py)

In [4]:
# snmp_web.py
from flask import Flask, render_template, request
from pysnmp.hlapi import *

app = Flask(__name__)

# Diccionario de OIDs comunes
OIDS = {
    'sysDescr': '1.3.6.1.2.1.1.1.0',
    'sysUptime': '1.3.6.1.2.1.1.3.0',
    'sysName': '1.3.6.1.2.1.1.5.0',
    'sysLocation': '1.3.6.1.2.1.1.6.0'
}

def snmp_get(ip, community, oid):
    iterator = getCmd(
        SnmpEngine(),
        CommunityData(community, mpModel=0),
        UdpTransportTarget((ip, 161)),
        ContextData(),
        ObjectType(ObjectIdentity(oid))
    )

    errorIndication, errorStatus, errorIndex, varBinds = next(iterator)

    if errorIndication:
        return f"Error: {errorIndication}"
    elif errorStatus:
        return f"Error: {errorStatus.prettyPrint()}"
    else:
        for varBind in varBinds:
            return str(varBind[1])

@app.route('/', methods=['GET', 'POST'])
def index():
    result = {}
    if request.method == 'POST':
        ip = request.form['ip']
        community = request.form['community']
        for name, oid in OIDS.items():
            value = snmp_get(ip, community, oid)
            result[name] = value
    return render_template('index.html', result=result)

if __name__ == '__main__':
    app.run(debug=True)


ModuleNotFoundError: No module named 'flask'

Plantilla HTML (templates/index.html)

##### Ampliamos la app Flask para mostrar más parámetros SNMP comunes

| Nombre           | OID                   | Descripción                              |
| ---------------- | --------------------- | ---------------------------------------- |
| `sysDescr`       | 1.3.6.1.2.1.1.1.0     | Descripción del sistema                  |
| `sysUptime`      | 1.3.6.1.2.1.1.3.0     | Tiempo en línea desde el último arranque |
| `sysName`        | 1.3.6.1.2.1.1.5.0     | Nombre del sistema                       |
| `sysLocation`    | 1.3.6.1.2.1.1.6.0     | Ubicación física                         |
| `ifNumber`       | 1.3.6.1.2.1.2.1.0     | Número de interfaces                     |
| `ifDescr.1`      | 1.3.6.1.2.1.2.2.1.2.1 | Nombre de la interfaz 1                  |
| `ifOperStatus.1` | 1.3.6.1.2.1.2.2.1.8.1 | Estado operativo de la interfaz 1        |


Modifico el diccionario OIDS en snmp_web.py.
Se pueden agregar más interfaces duplicando el patrón, cambiando 1 por 2, 3, etc.

In [ ]:
OIDS = {
    'sysDescr': '1.3.6.1.2.1.1.1.0',
    'sysUptime': '1.3.6.1.2.1.1.3.0',
    'sysName': '1.3.6.1.2.1.1.5.0',
    'sysLocation': '1.3.6.1.2.1.1.6.0',
    'ifNumber': '1.3.6.1.2.1.2.1.0',
    'ifDescr_1': '1.3.6.1.2.1.2.2.1.2.1',
    'ifOperStatus_1': '1.3.6.1.2.1.2.2.1.8.1'
}

- Convertir valores binarios como ifOperStatus
- Traducir estados (por ejemplo 1 = up, 2 = down) en la función snmp_get()

In [ ]:
def snmp_get(ip, community, oid):
    iterator = getCmd(
        SnmpEngine(),
        CommunityData(community, mpModel=0),
        UdpTransportTarget((ip, 161)),
        ContextData(),
        ObjectType(ObjectIdentity(oid))
    )

    errorIndication, errorStatus, errorIndex, varBinds = next(iterator)

    if errorIndication:
        return f"Error: {errorIndication}"
    elif errorStatus:
        return f"Error: {errorStatus.prettyPrint()}"
    else:
        for varBind in varBinds:
            val = str(varBind[1])
            if '2.2.1.8' in oid:  # OperStatus
                val = {'1': 'up', '2': 'down', '3': 'testing'}.get(val, val)
            return val

##### Se crea una nueva MIB para ejemplificar la estructura.
Una MIB es un archivo de texto que define objetos SNMP estructurados jerárquicamente. Se puede crear un módulo OIDs propios, por ejemplo para simular sensores, dispositivos, etc.  
Una MIB (Management Information Base) es un archivo que define objetos que pueden ser gestionados en un dispositivo mediante el protocolo SNMP (Simple Network Management Protocol). Las MIBs permiten a las herramientas de monitoreo (como Zabbix, Nagios, PRTG, etc.) entender qué datos puede consultar o controlar en un dispositivo de red (como un router, switch, servidor, etc.).   
Es un archivo de texto escrito en un lenguaje formal basado en ASN.1 (Abstract Syntax Notation One). 
Este lenguaje define:
- Objetos gestionables
- Nombres simbólicos
- Tipos de datos
- Ubicaciones dentro de una jerarquía de objetos (OID)

**Encabezado del módulo**   
Define el nombre del módulo y sus dependencias.
```json
MY-MIB DEFINITIONS ::= BEGIN

IMPORTS
    MODULE-IDENTITY, OBJECT-TYPE, Integer32
        FROM SNMPv2-SMI
    TEXTUAL-CONVENTION
        FROM SNMPv2-TC;
```
**Identidad del módulo (MODULE-IDENTITY)**
Define quién creó la MIB y otra información administrativa.
```json
myMIB MODULE-IDENTITY
    LAST-UPDATED "202405190000Z"
    ORGANIZATION "Mi Empresa"
    CONTACT-INFO "soporte@miempresa.com"
    DESCRIPTION "MIB personalizada para sensores de invernadero."
    ::= { enterprises 99999 }
```
99999 es un OID asignado a la empresa por IANA, o un número privado en pruebas.

**Definiciones de objetos (OBJECT-TYPE)**
Cada objeto representa una métrica o configuración accesible por SNMP.
```json
temperaturaSensor OBJECT-TYPE
    SYNTAX      Integer32
    MAX-ACCESS  read-only
    STATUS      current
    DESCRIPTION "Temperatura medida por el sensor en grados Celsius."
    ::= { myMIB 1 }

```
- SYNTAX: tipo de dato (Integer32, OCTET STRING, etc.)
- MAX-ACCESS: read-only, read-write, not-accessible
- STATUS: current, deprecated, obsolete
- ::= { myMIB 1 }: asigna un OID relativo al módulo

**Jerarquía de OIDs (Object Identifiers)**
Los OIDs son rutas jerárquicas, como direcciones, que identifican objetos.
```text
1.3.6.1.4.1.99999.1
│ │ │ │ │ └──── Empresa privada (enterprise ID asignado por IANA)
│ │ │ │ └────── Internet privado
│ │ │ └──────── MIB-II
│ │ └────────── ISO Identificado por SNMP
│ └──────────── Organización internacional
└────────────── ISO
```
  
**Estructura mínima de una MIB**

```json
MY-DEMO-MIB DEFINITIONS ::= BEGIN

IMPORTS
    MODULE-IDENTITY, OBJECT-TYPE, enterprises FROM SNMPv2-SMI;

myDemoMIB MODULE-IDENTITY
    LAST-UPDATED "202505120000Z"
    ORGANIZATION "Nombre o Institución"
    CONTACT-INFO "un.email@dominio.com"
    DESCRIPTION "MIB de ejemplo."
    ::= { enterprises 99999 }

-- Definimos un nodo llamado demoText

demoText OBJECT-TYPE
    SYNTAX      OCTET STRING
    MAX-ACCESS  read-write
    STATUS      current
    DESCRIPTION "Texto de prueba"
    ::= { myDemoMIB 1 }

END
```

```json
MY-SENSOR-MIB DEFINITIONS ::= BEGIN

IMPORTS
    MODULE-IDENTITY, OBJECT-TYPE, Integer32
        FROM SNMPv2-SMI;

mySensorMIB MODULE-IDENTITY
    LAST-UPDATED "202405190000Z"
    ORGANIZATION "Mi Empresa"
    DESCRIPTION "MIB personalizada para sensores de temperatura."
    ::= { enterprises 99999 }

temperatura OBJECT-TYPE
    SYNTAX      Integer32
    MAX-ACCESS  read-only
    STATUS      current
    DESCRIPTION "Temperatura en grados Celsius."
    ::= { mySensorMIB 1 }

humedad OBJECT-TYPE
    SYNTAX      Integer32
    MAX-ACCESS  read-only
    STATUS      current
    DESCRIPTION "Humedad relativa en porcentaje."
    ::= { mySensorMIB 2 }

END
```

#### MIB para un sistema IoT simulado, con sensores de temperatura, humedad y un relé de riego,  con soporte de lectura y escritura para pruebas con SNMP.

MIB: IOT-DEMO-MIB.txt   
Guardar este archivo:

```json
IOT-DEMO-MIB DEFINITIONS ::= BEGIN

IMPORTS
    MODULE-IDENTITY, OBJECT-TYPE, Integer32, enterprises FROM SNMPv2-SMI;

iotDemoMIB MODULE-IDENTITY
    LAST-UPDATED "202505120000Z"
    ORGANIZATION "Demo SNMP IoT"
    CONTACT-INFO "iot@educacion.demo"
    DESCRIPTION "MIB educativa que simula sensores IoT en un invernadero"
    ::= { enterprises 55555 }  -- Puedes cambiar por tu PEN si lo tienes

-- Nodo principal
iotDevices       OBJECT IDENTIFIER ::= { iotDemoMIB 1 }

-- Sensor de temperatura (en °C)
temperature OBJECT-TYPE
    SYNTAX      Integer32
    MAX-ACCESS  read-only
    STATUS      current
    DESCRIPTION "Sensor de temperatura en grados Celsius"
    ::= { iotDevices 1 }

-- Sensor de humedad (%)
humidity OBJECT-TYPE
    SYNTAX      Integer32
    MAX-ACCESS  read-only
    STATUS      current
    DESCRIPTION "Sensor de humedad relativa en porcentaje"
    ::= { iotDevices 2 }

-- Relay de riego (0 = off, 1 = on)
irrigationRelay OBJECT-TYPE
    SYNTAX      Integer32
    MAX-ACCESS  read-write
    STATUS      current
    DESCRIPTION "Estado del relé de riego: 0=OFF, 1=ON"
    ::= { iotDevices 3 }

END
```

##### Integrar esta MIB en la app Flask y cambiar los valores desde una interfaz web
Integrar IOT-DEMO-MIB en una app Flask:
- Consultar sensores SNMP (temperatura, humedad)
- Activar/desactivar un relay de riego (con snmpset)
- Estructura del proyecto
```text
iot_snmp_app/
├── app.py
├── templates/
│   └── index.html
├── static/
│   └── style.css (opcional)
└── IOT-DEMO-MIB.txt
```

In [ ]:
# app.py (Flask + pysnmp)
from flask import Flask, render_template, request, redirect, url_for
from pysnmp.hlapi import *

app = Flask(__name__)

TARGET = 'localhost'  # O IP de tu dispositivo SNMP
COMMUNITY = 'public'
MIB_MODULE = 'IOT-DEMO-MIB'

OIDS = {
    'temperature': ('IOT-DEMO-MIB', 'temperature'),
    'humidity': ('IOT-DEMO-MIB', 'humidity'),
    'irrigationRelay': ('IOT-DEMO-MIB', 'irrigationRelay')
}

def snmp_get(name):
    oid = ObjectIdentity(*OIDS[name]).addMibSource('/usr/share/snmp/mibs')
    iterator = getCmd(
        SnmpEngine(),
        CommunityData(COMMUNITY),
        UdpTransportTarget((TARGET, 161)),
        ContextData(),
        ObjectType(oid)
    )
    errorIndication, errorStatus, _, varBinds = next(iterator)
    if errorIndication or errorStatus:
        return "Error"
    return str(varBinds[0][1])

def snmp_set(name, value):
    oid = ObjectIdentity(*OIDS[name]).addMibSource('/usr/share/snmp/mibs')
    iterator = setCmd(
        SnmpEngine(),
        CommunityData(COMMUNITY),
        UdpTransportTarget((TARGET, 161)),
        ContextData(),
        ObjectType(oid, Integer(int(value)))
    )
    errorIndication, errorStatus, _, varBinds = next(iterator)
    if errorIndication or errorStatus:
        return "Error"
    return str(varBinds[0][1])

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        estado = request.form.get('relay')
        snmp_set('irrigationRelay', estado)
        return redirect(url_for('index'))

    temperature = snmp_get('temperature')
    humidity = snmp_get('humidity')
    relay = snmp_get('irrigationRelay')

    return render_template('index.html', temp=temperature, hum=humidity, relay=relay)

if __name__ == '__main__':
    app.run(debug=True)


HTML: templates/index.html

```html
<!DOCTYPE html>
<html>
<head>
    <title>SNMP IoT Dashboard</title>
</head>
<body>
    <h1>Estado del Invernadero (SNMP)</h1>
    <p><strong>Temperatura:</strong> {{ temp }} °C</p>
    <p><strong>Humedad:</strong> {{ hum }} %</p>
    <p><strong>Riego:</strong> {% if relay == '1' %}ACTIVO{% else %}INACTIVO{% endif %}</p>

    <form method="post">
        <label for="relay">Cambiar estado de riego:</label>
        <select name="relay">
            <option value="1" {% if relay == '1' %}selected{% endif %}>Activar</option>
            <option value="0" {% if relay == '0' %}selected{% endif %}>Desactivar</option>
        </select>
        <button type="submit">Aplicar</button>
    </form>
</body>
</html>
```

##### Crear o extender una mib para que SNMP pueda ver el espacio libre en un disco bajo ubuntu


**Usamos MIB estándar HOST-RESOURCES-MIB (más fácil y portable)**

- MIB que reporta uso de disco y memoria: HOST-RESOURCES-MIB.
- Instalar y configura snmpd en Ubuntu:
```text
sudo apt update
sudo apt install snmpd
sudo nano /etc/snmp/snmpd.conf
rocommunity public 127.0.0.1
agentAddress udp:161

# Activar el soporte para HOST-RESOURCES-MIB:
sudo systemctl restart snmpd
# Prueba
snmpwalk -v2c -c public localhost HOST-RESOURCES-MIB::hrStorageDescr
# mostrará entradas como:
HOST-RESOURCES-MIB::hrStorageDescr.31 = STRING: /home

# Buscar la partición deseada (/, /home, etc.) y consultar:

snmpwalk -v2c -c public localhost HOST-RESOURCES-MIB::hrStorageUsed.31
snmpwalk -v2c -c public localhost HOST-RESOURCES-MIB::hrStorageSize.31
# Con esos dos datos se puede calcular el uso y espacio libre en disco.
```

**Crear una MIB personalizada + script para reportar espacio libre**    
**Exponer el % de espacio libre en disco / como OID .1.3.6.1.4.1.55556.1.1.0**    
- Guardar este archivo como DISK-USAGE-MIB.txt (puede ir en /usr/share/snmp/mibs/):
```json
DISK-USAGE-MIB DEFINITIONS ::= BEGIN

IMPORTS
    MODULE-IDENTITY, OBJECT-TYPE, Integer32, enterprises FROM SNMPv2-SMI;

diskUsageMIB MODULE-IDENTITY
    LAST-UPDATED "202505170000Z"
    ORGANIZATION "MIB educativa disco"
    ::= { enterprises 55556 }

diskInfo OBJECT IDENTIFIER ::= { diskUsageMIB 1 }

diskFreePercent OBJECT-TYPE
    SYNTAX      Integer32
    MAX-ACCESS  read-only
    STATUS      current
    DESCRIPTION "Porcentaje de espacio libre en /"
    ::= { diskInfo 1 }

END
```


Creo script: /usr/local/bin/snmp_disk_persist.py

In [ ]:
#!/usr/bin/env python3
import os
import sys

OID_DISK_FREE = '.1.3.6.1.4.1.55556.1.1.0'

def get_disk_free_percent():
    statvfs = os.statvfs('/')
    total = statvfs.f_blocks * statvfs.f_frsize
    free = statvfs.f_bfree * statvfs.f_frsize
    percent = int((free / total) * 100)
    return percent

def main():
    print("PASS_PERSIST")
    sys.stdout.flush()
    
    while True:
        line = sys.stdin.readline().strip()
        if not line:
            break

        if line == "PING":
            print("PONG")
        elif line == f"getnext":
            print(f"{OID_DISK_FREE}")
            print("integer")
            print(get_disk_free_percent())
        elif line == f"get {OID_DISK_FREE}":
            print("integer")
            print(get_disk_free_percent())
        else:
            print("NONE")

        sys.stdout.flush()

if __name__ == "__main__":
    main()
